In [2]:

"""
uninformed_search_lab.py

Contains:
- bfs(graph, start, goal)           : unweighted shortest-path BFS for mazes/graphs
- dfs_recursive(graph, start, goal) : recursive DFS (first-found path)
- bfs_8puzzle(start, goal)         : BFS for the 8-puzzle (uses tuple state)
- ucs(graph, start, goal)          : Uniform Cost Search (using heapq)
- compare_bfs_dfs(...)             : simple comparison instrumentation

Simple examples at the bottom show how to use each function.
"""

from collections import deque
import heapq
import time
from typing import Dict, List, Tuple, Optional, Set

# ---------------------------
# Common helpers
# ---------------------------

def reconstruct_path(parent: dict, goal):
    """
    parent: dict[state] = (prev_state, action) or None
    goal: goal state
    returns list of states from start to goal (inclusive) and list of actions
    """
    states = []
    actions = []
    cur = goal
    while cur is not None:
        prev_action = parent[cur]  # either None or (prev_state, action)
        if prev_action is None:
            states.append(cur)
            break
        prev, action = prev_action
        states.append(cur)
        actions.append(action)
        cur = prev
    states.reverse()
    actions.reverse()
    return states, actions

# ---------------------------
# Experiment 1: BFS (unweighted)
# ---------------------------

def bfs(graph: Dict, start, goal):
    """
    Simple BFS for unweighted graph.
    graph: dict[node] -> list of neighbors
    returns: (path_states, actions, nodes_expanded, max_frontier_size, time_elapsed)
    actions are simply neighbor nodes (or you can store move names)
    """
    t0 = time.time()
    frontier = deque([start])
    parent = {start: None}   # store None for start, and (prev_state, action) for others
    nodes_expanded = 0
    max_frontier_size = 1

    while frontier:
        node = frontier.popleft()
        nodes_expanded += 1
        if node == goal:
            path_states, actions = reconstruct_path(parent, goal)
            return path_states, actions, nodes_expanded, max_frontier_size, time.time()-t0

        for nb in graph.get(node, []):
            if nb not in parent:
                parent[nb] = (node, nb)   # action is "go to nb"
                frontier.append(nb)
                if len(frontier) > max_frontier_size:
                    max_frontier_size = len(frontier)

    return None, None, nodes_expanded, max_frontier_size, time.time()-t0

# ---------------------------
# Experiment 2: Recursive DFS (first-found)
# ---------------------------

def dfs_recursive(graph: Dict, start, goal, max_depth_limit: int = 10000):
    """
    Recursive DFS that returns the first path found from start to goal.
    Uses path and path_set to avoid cycles in current path.
    Also tracks nodes_expanded and maximum recursion depth (path length).
    """
    t0 = time.time()
    nodes_expanded = 0
    max_depth_seen = 1

    def dfs_helper(current, path, path_set):
        nonlocal nodes_expanded, max_depth_seen
        nodes_expanded += 1
        if len(path) > max_depth_seen:
            max_depth_seen = len(path)
        if current == goal:
            return True
        if len(path) >= max_depth_limit:
            return False  # avoid infinite recursion on huge graphs
        for nb in graph.get(current, []):
            if nb in path_set:
                continue
            path.append(nb)
            path_set.add(nb)
            found = dfs_helper(nb, path, path_set)
            if found:
                return True
            path.pop()
            path_set.remove(nb)
        return False

    path = [start]
    path_set = {start}
    found = dfs_helper(start, path, path_set)
    time_elapsed = time.time() - t0
    if found:
        # convert to parent-like actions for uniform output: actions are nodes themselves
        actions = [(path[i+1]) for i in range(len(path)-1)]
        return path, actions, nodes_expanded, max_depth_seen, time_elapsed
    else:
        return None, None, nodes_expanded, max_depth_seen, time_elapsed

# ---------------------------
# Experiment 3: 8-puzzle BFS
# ---------------------------

# Helper constants for 3x3 puzzle
PUZZLE_SIZE = 3  # 3x3
GOAL_STATE = (1,2,3,4,5,6,7,8,0)  # canonical goal (0 is blank)

def idx_to_rc(idx: int) -> Tuple[int,int]:
    return divmod(idx, PUZZLE_SIZE)

def rc_to_idx(r:int, c:int) -> int:
    return r * PUZZLE_SIZE + c

def get_8puzzle_neighbors(state: Tuple[int,...]) -> List[Tuple[str, Tuple[int,...]]]:
    """
    Returns list of (action_name, next_state). Action is 'Up', 'Down', 'Left', 'Right'.
    state is a tuple of length 9.
    """
    neighbors = []
    zero_idx = state.index(0)
    zr, zc = idx_to_rc(zero_idx)

    moves = []
    if zr > 0: moves.append((-1, 0, "Up"))
    if zr < PUZZLE_SIZE-1: moves.append((1, 0, "Down"))
    if zc > 0: moves.append((0, -1, "Left"))
    if zc < PUZZLE_SIZE-1: moves.append((0, 1, "Right"))

    for dr, dc, name in moves:
        nr, nc = zr + dr, zc + dc
        swap_idx = rc_to_idx(nr, nc)
        new_list = list(state)
        new_list[zero_idx], new_list[swap_idx] = new_list[swap_idx], new_list[zero_idx]
        neighbors.append((name, tuple(new_list)))
    return neighbors

def is_solvable_8puzzle(state: Tuple[int,...]) -> bool:
    """
    Solvability test for 3x3 8-puzzle: parity of inversions must match goal.
    For 3x3, puzzle is solvable if number of inversions is even.
    """
    arr = [x for x in state if x != 0]
    inv = 0
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return (inv % 2) == 0

def bfs_8puzzle(start: Tuple[int,...], goal: Tuple[int,...] = GOAL_STATE):
    """
    BFS specialized for 8-puzzle.
    Returns: (path_states, actions, nodes_expanded, max_frontier_size, time_elapsed)
    """
    if start == goal:
        return [start], [], 0, 1, 0.0
    if not is_solvable_8puzzle(start):
        return None, None, 0, 0, 0.0

    t0 = time.time()
    frontier = deque([start])
    parent = {start: None}
    nodes_expanded = 0
    max_frontier = 1

    while frontier:
        s = frontier.popleft()
        nodes_expanded += 1
        if s == goal:
            path_states, actions = reconstruct_path(parent, goal)
            return path_states, actions, nodes_expanded, max_frontier, time.time()-t0
        for action, nxt in get_8puzzle_neighbors(s):
            if nxt not in parent:
                parent[nxt] = (s, action)
                frontier.append(nxt)
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)
    return None, None, nodes_expanded, max_frontier, time.time()-t0

# ---------------------------
# Experiment 4: Uniform Cost Search (UCS)
# ---------------------------

def ucs(graph_weighted: Dict, start, goal):
    """
    Uniform Cost Search using heapq (min-heap).
    graph_weighted: dict[node] -> list of (neighbor, cost)
    Returns: (path_states, actions, total_cost, nodes_expanded, max_frontier_size, time_elapsed)
    """
    t0 = time.time()
    frontier = []  # heap of tuples (cost, counter, node)
    counter = 0
    heapq.heappush(frontier, (0, counter, start))
    parent = {start: None}
    cost_so_far = {start: 0}
    nodes_expanded = 0
    max_frontier = 1

    while frontier:
        cost, _, node = heapq.heappop(frontier)
        # If this popped state has a cost > best known, skip it (stale)
        if cost > cost_so_far.get(node, float('inf')):
            continue
        nodes_expanded += 1
        if node == goal:
            path_states, actions = reconstruct_path(parent, goal)
            return path_states, actions, cost, nodes_expanded, max_frontier, time.time()-t0

        for nb, step_cost in graph_weighted.get(node, []):
            new_cost = cost + step_cost
            if nb not in cost_so_far or new_cost < cost_so_far[nb]:
                cost_so_far[nb] = new_cost
                counter += 1
                heapq.heappush(frontier, (new_cost, counter, nb))
                parent[nb] = (node, f"{node}->{nb} ({step_cost})")
                if len(frontier) > max_frontier:
                    max_frontier = len(frontier)

    return None, None, float('inf'), nodes_expanded, max_frontier, time.time()-t0

# ---------------------------
# Experiment 5: Compare BFS vs DFS instrumentation
# ---------------------------

def compare_bfs_dfs(graph: Dict, start, goal):
    """
    Runs BFS and recursive DFS on same graph and prints a short comparison.
    """
    print("Running BFS...")
    bfs_res = bfs(graph, start, goal)
    print("Running DFS (recursive)...")
    dfs_res = dfs_recursive(graph, start, goal)

    print("\n--- Results ---")
    if bfs_res[0] is not None:
        path_states, actions, nodes_expanded, max_frontier, t = bfs_res
        print(f"BFS: path length (states) = {len(path_states)}, nodes_expanded = {nodes_expanded}, max_frontier = {max_frontier}, time = {t:.6f}s")
        print("  path:", path_states)
    else:
        print("BFS: No path found")

    if dfs_res[0] is not None:
        path_states, actions, nodes_expanded, max_depth, t = dfs_res
        print(f"DFS: path length (states) = {len(path_states)}, nodes_expanded = {nodes_expanded}, max_depth_seen = {max_depth}, time = {t:.6f}s")
        print("  path:", path_states)
    else:
        print("DFS: No path found")

# ---------------------------
# Example usage / small tests
# ---------------------------

if __name__ == "__main__":
    # Simple graph (unweighted) for BFS and DFS examples
    sample_graph = {
        'A': ['B', 'C'],
        'B': ['D', 'E'],
        'C': ['F'],
        'D': ['G'],
        'E': ['G'],
        'F': ['E'],
        'G': []
    }

    print("=== BFS example on small graph ===")
    path, actions, nodes_exp, max_frontier, t = bfs(sample_graph, 'A', 'G')
    print("BFS path:", path)
    print("Nodes expanded:", nodes_exp, "Max frontier:", max_frontier, "Time:", f"{t:.6f}s")
    print()

    print("=== DFS (recursive) example on same graph ===")
    path_dfs, actions_dfs, nodes_exp_dfs, max_depth, tdfs = dfs_recursive(sample_graph, 'A', 'G')
    print("DFS path (first found):", path_dfs)
    print("Nodes expanded:", nodes_exp_dfs, "Max depth seen:", max_depth, "Time:", f"{tdfs:.6f}s")
    print()

    print("=== Compare BFS vs DFS on sample graph ===")
    compare_bfs_dfs(sample_graph, 'A', 'G')
    print()

    # 8-puzzle example (small scramble)
    print("=== 8-puzzle BFS example ===")
    start_8 = (1,2,3,4,5,6,0,7,8)  # blank near end: solvable and few moves away
    print("Start state:", start_8)
    solvable = is_solvable_8puzzle(start_8)
    print("Is solvable?", solvable)
    if solvable:
        p_states, p_actions, nodes_e, max_f, t = bfs_8puzzle(start_8, GOAL_STATE)
        if p_states:
            print("8-puzzle BFS succeeded!")
            print("Moves:", p_actions)
            print("Number of states in solution path:", len(p_states))
            print("Nodes expanded:", nodes_e, "Max frontier:", max_f, "Time:", f"{t:.6f}s")
        else:
            print("8-puzzle BFS: no solution found (unexpected).")
    print()

    # UCS example (weighted graph)
    print("=== UCS example (weighted graph) ===")
    weighted_graph = {
        'A': [('B', 50), ('C', 10)],
        'B': [('D', 20), ('G', 100)],
        'C': [('F', 15)],
        'D': [('G', 10)],
        'F': [('G', 20)],
        'G': []
    }
    # Expected optimal A->C->F->G cost = 10 + 15 + 20 = 45
    path_u, actions_u, total_cost, nodes_e, max_f, t = ucs(weighted_graph, 'A', 'G')
    print("UCS path:", path_u)
    print("Total cost:", total_cost)
    print("Nodes expanded:", nodes_e, "Max frontier:", max_f, "Time:", f"{t:.6f}s")
    print()


=== BFS example on small graph ===
BFS path: ['A', 'B', 'D', 'G']
Nodes expanded: 7 Max frontier: 3 Time: 0.000014s

=== DFS (recursive) example on same graph ===
DFS path (first found): ['A', 'B', 'D', 'G']
Nodes expanded: 4 Max depth seen: 4 Time: 0.000006s

=== Compare BFS vs DFS on sample graph ===
Running BFS...
Running DFS (recursive)...

--- Results ---
BFS: path length (states) = 4, nodes_expanded = 7, max_frontier = 3, time = 0.000006s
  path: ['A', 'B', 'D', 'G']
DFS: path length (states) = 4, nodes_expanded = 4, max_depth_seen = 4, time = 0.000002s
  path: ['A', 'B', 'D', 'G']

=== 8-puzzle BFS example ===
Start state: (1, 2, 3, 4, 5, 6, 0, 7, 8)
Is solvable? True
8-puzzle BFS succeeded!
Moves: ['Right', 'Right']
Number of states in solution path: 3
Nodes expanded: 7 Max frontier: 8 Time: 0.000024s

=== UCS example (weighted graph) ===
UCS path: ['A', 'C', 'F', 'G']
Total cost: 45
Nodes expanded: 4 Max frontier: 2 Time: 0.000014s



# method 2

In [3]:
# ai_lab_uninformed_search.py
# All Uninformed Search Algorithms in ONE file
# BFS, DFS, 8-Puzzle BFS, Uniform Cost Search, Comparison

from collections import deque
import heapq
import time
from pprint import pprint

# -------------------------------------------------
# PATH RECONSTRUCTION
# -------------------------------------------------

def build_path(parent, goal):
    path = []
    while goal is not None:
        path.append(goal)
        goal = parent.get(goal)
    return path[::-1]


# -------------------------------------------------
# BFS (Unweighted Graph / Maze)
# -------------------------------------------------

def bfs(graph, start, goal):
    q = deque([start])
    parent = {start: None}
    nodes = 0

    while q:
        node = q.popleft()
        nodes += 1

        if node == goal:
            return build_path(parent, goal), nodes

        for nb in graph[node]:
            if nb not in parent:
                parent[nb] = node
                q.append(nb)

    return None, nodes


# -------------------------------------------------
# DFS (Recursive)
# -------------------------------------------------

def dfs(graph, node, goal, visited, path):
    visited.add(node)
    path.append(node)

    if node == goal:
        return True

    for nb in graph[node]:
        if nb not in visited:
            if dfs(graph, nb, goal, visited, path):
                return True

    path.pop()
    return False


def run_dfs(graph, start, goal):
    visited = set()
    path = []
    dfs(graph, start, goal, visited, path)
    return path


# -------------------------------------------------
# 8 PUZZLE BFS
# -------------------------------------------------

def neighbors_8puzzle(state):
    res = []
    i = state.index(0)
    r, c = divmod(i, 3)

    moves = []
    if r > 0: moves.append(-3)
    if r < 2: moves.append(3)
    if c > 0: moves.append(-1)
    if c < 2: moves.append(1)

    for m in moves:
        s = list(state)
        j = i + m
        s[i], s[j] = s[j], s[i]
        res.append(tuple(s))

    return res


def bfs_8puzzle(start, goal):
    q = deque([start])
    parent = {start: None}

    while q:
        s = q.popleft()

        if s == goal:
            return build_path(parent, goal)

        for nb in neighbors_8puzzle(s):
            if nb not in parent:
                parent[nb] = s
                q.append(nb)

    return None


# -------------------------------------------------
# UNIFORM COST SEARCH
# -------------------------------------------------

def ucs(graph, start, goal):
    pq = [(0, start)]
    parent = {start: None}
    cost = {start: 0}

    while pq:
        g, node = heapq.heappop(pq)

        if node == goal:
            return build_path(parent, goal), g

        for nb, w in graph[node]:
            new_cost = g + w

            if nb not in cost or new_cost < cost[nb]:
                cost[nb] = new_cost
                parent[nb] = node
                heapq.heappush(pq, (new_cost, nb))

    return None, float("inf")


# -------------------------------------------------
# PERFORMANCE COMPARISON
# -------------------------------------------------

def compare(graph, start, goal):
    print("\n--- PERFORMANCE COMPARISON ---")

    t1 = time.time()
    p1, n1 = bfs(graph, start, goal)
    t2 = time.time()

    t3 = time.time()
    p2 = run_dfs(graph, start, goal)
    t4 = time.time()

    print("BFS Path:", p1)
    print("BFS Nodes:", n1, "Time:", round(t2-t1,6))

    print("DFS Path:", p2)
    print("DFS Time:", round(t4-t3,6))


# -------------------------------------------------
# MAIN PROGRAM
# -------------------------------------------------

if __name__ == "__main__":

    graph = {
        'A':['B','C'],
        'B':['D','E'],
        'C':['F'],
        'D':['G'],
        'E':['G'],
        'F':[],
        'G':[]
    }

    weighted_graph = {
        'A':[('B',50),('C',10)],
        'B':[('D',20),('G',100)],
        'C':[('F',15)],
        'D':[('G',10)],
        'F':[('G',20)],
        'G':[]
    }

    print("\n=== BFS ===")
    print(bfs(graph,'A','G'))

    print("\n=== DFS ===")
    print(run_dfs(graph,'A','G'))

    print("\n=== UCS ===")
    print(ucs(weighted_graph,'A','G'))

    print("\n=== 8 PUZZLE ===")
    start = (1,2,3,4,5,6,0,7,8)
    goal  = (1,2,3,4,5,6,7,8,0)
    pprint(bfs_8puzzle(start,goal))

    compare(graph,'A','G')



=== BFS ===
(['A', 'B', 'D', 'G'], 7)

=== DFS ===
['A', 'B', 'D', 'G']

=== UCS ===
(['A', 'C', 'F', 'G'], 45)

=== 8 PUZZLE ===
[(1, 2, 3, 4, 5, 6, 0, 7, 8),
 (1, 2, 3, 4, 5, 6, 7, 0, 8),
 (1, 2, 3, 4, 5, 6, 7, 8, 0)]

--- PERFORMANCE COMPARISON ---
BFS Path: ['A', 'B', 'D', 'G']
BFS Nodes: 7 Time: 6e-06
DFS Path: ['A', 'B', 'D', 'G']
DFS Time: 3e-06


In [4]:
"""
lab4_with_libs.py
Lab 4: Greedy Best-First, A*, RBFS — library-first, fallback-safe.
Run: python lab4_with_libs.py
Optional pip installs for nicer output: pip install networkx psutil rich tqdm
"""

import time
import heapq
from collections import deque
from functools import lru_cache

# Try to import helpful third-party libs, otherwise fallback
try:
    import networkx as nx
    HAS_NX = True
except Exception:
    HAS_NX = False

try:
    import psutil
    HAS_PSUTIL = True
except Exception:
    HAS_PSUTIL = False

try:
    from rich.pretty import pprint as rprint
    from rich import print as rprint_line
    HAS_RICH = True
except Exception:
    HAS_RICH = False

def maybe_print(*args, **kwargs):
    if HAS_RICH:
        rprint_line(*args, **kwargs)
    else:
        print(*args, **kwargs)

# -----------------------
# Utilities (common)
# -----------------------

def reconstruct(parent, goal):
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = parent.get(cur)
    path.reverse()
    return path

def time_and_mem():
    t = time.time()
    mem = None
    if HAS_PSUTIL:
        process = psutil.Process()
        mem = process.memory_info().rss  # bytes
    return t, mem

# -----------------------
# Grid helpers
# -----------------------

def grid_neighbors(pos, grid):
    r,c = pos
    rows, cols = len(grid), len(grid[0])
    for dr,dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        nr, nc = r+dr, c+dc
        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == 0:
            yield (nr,nc), 1

def manhattan(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

# -----------------------
# Greedy Best-First Search
# Uses networkx if available for clean code on graphs, otherwise simple heap-based.
# -----------------------

def greedy_bfs(start, goal, successors_fn, heuristic_fn):
    t0 = time.time()
    nodes_expanded = 0
    open_heap = [(heuristic_fn(start), start)]
    parent = {start: None}
    closed = set()

    while open_heap:
        h, node = heapq.heappop(open_heap)
        if node in closed:
            continue
        closed.add(node)
        nodes_expanded += 1
        if node == goal:
            return {'path': reconstruct(parent, goal), 'nodes': nodes_expanded, 'time': time.time()-t0}
        for nb, cost in successors_fn(node):
            if nb not in closed and nb not in parent:
                parent[nb] = node
                heapq.heappush(open_heap, (heuristic_fn(nb), nb))
    return {'path': None, 'nodes': nodes_expanded, 'time': time.time()-t0}

# -----------------------
# A* search
# If networkx available and states are hashable nodes, use nx.astar_path for succinctness.
# Otherwise fallback to standard heap-based A*.
# -----------------------

def a_star(start, goal, successors_fn, heuristic_fn):
    # If networkx is available and start/goal are suitable node types and successors_fn maps to neighbors, we skip.
    # But safer to use our generic A* (works always).
    t0 = time.time()
    open_heap = []
    counter = 0
    heapq.heappush(open_heap, (heuristic_fn(start), counter, start))
    g = {start: 0}
    parent = {start: None}
    closed = set()
    nodes_expanded = 0

    while open_heap:
        f, _, node = heapq.heappop(open_heap)
        if node in closed:
            continue
        closed.add(node)
        nodes_expanded += 1
        if node == goal:
            return {'path': reconstruct(parent, goal), 'cost': g[node], 'nodes': nodes_expanded, 'time': time.time()-t0}
        for nb, cost in successors_fn(node):
            ng = g[node] + cost
            if nb not in g or ng < g[nb]:
                g[nb] = ng
                parent[nb] = node
                counter += 1
                heapq.heappush(open_heap, (ng + heuristic_fn(nb), counter, nb))
    return {'path': None, 'nodes': nodes_expanded, 'time': time.time()-t0}

# -----------------------
# RBFS (recursive best-first)
# Simple, clear RBFS implementation (uses heuristic_fn, successors_fn)
# -----------------------

def rbfs(start, goal, successors_fn, heuristic_fn):
    t0 = time.time()
    parent = {start: None}
    nodes_expanded = 0

    def rbfs_rec(node, g, f_limit):
        nonlocal nodes_expanded
        nodes_expanded += 1
        f_node = g + heuristic_fn(node)
        if node == goal:
            return True, g, f_node
        children = []
        for nb, cost in successors_fn(node):
            ng = g + cost
            children.append([nb, ng, ng + heuristic_fn(nb)])  # [node,g,f]
            parent.setdefault(nb, node)
        if not children:
            return False, float('inf'), float('inf')
        children.sort(key=lambda x: x[2])
        while True:
            best = children[0]
            if best[2] > f_limit:
                return False, float('inf'), best[2]
            alt = children[1][2] if len(children) > 1 else float('inf')
            found, cost_res, best_f = rbfs_rec(best[0], best[1], min(f_limit, alt))
            children[0][2] = best_f
            children.sort(key=lambda x: x[2])
            if found:
                return True, cost_res, best_f

    found, cost, fval = rbfs_rec(start, 0, float('inf'))
    return {'path': reconstruct(parent, goal) if found else None, 'cost': cost if found else None, 'nodes': nodes_expanded, 'time': time.time()-t0}

# -----------------------
# 8-puzzle helpers (A* test and RBFS test)
# -----------------------

PUZZLE_GOAL = (1,2,3,4,5,6,7,8,0)

def neighbors_8puzzle(state):
    idx = state.index(0)
    r,c = divmod(idx, 3)
    moves = []
    if r>0: moves.append(-3)
    if r<2: moves.append(3)
    if c>0: moves.append(-1)
    if c<2: moves.append(1)
    for m in moves:
        s = list(state)
        j = idx + m
        s[idx], s[j] = s[j], s[idx]
        yield tuple(s), 1

def manhattan_8p(state, goal=PUZZLE_GOAL):
    total=0
    for i,v in enumerate(state):
        if v==0: continue
        tr,tc = divmod(i,3)
        gi = goal.index(v)
        gr,gc = divmod(gi,3)
        total += abs(tr-gr)+abs(tc-gc)
    return total

# -----------------------
# If networkx is installed, show how to use it for grid A* concisely
# -----------------------

def demo_with_networkx_grid(grid, start, goal):
    if not HAS_NX:
        maybe_print("networkx not available — skipping networkx demo.")
        return
    # create a graph where nodes are (r,c) and edges for free cells
    G = nx.Graph()
    rows, cols = len(grid), len(grid[0])
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 0:
                G.add_node((r,c))
    for node in list(G.nodes):
        for nb, cost in grid_neighbors(node, grid):
            G.add_edge(node, nb, weight=cost)
    # use networkx.astar_path
    path = nx.astar_path(G, start, goal, heuristic=lambda a,b: manhattan(a,b), weight='weight')
    cost = nx.path_weight(G, path, weight='weight') if hasattr(nx, 'path_weight') else sum(1 for _ in path)-1
    return path, cost

# -----------------------
# Example runs
# -----------------------

if __name__ == "__main__":
    maybe_print("=== Lab 4: library-friendly informed searches ===")
    grid = [
        [0,0,0,1,0],
        [1,1,0,1,0],
        [0,0,0,0,0],
        [0,1,1,1,0],
        [0,0,0,0,0]
    ]
    start = (0,0)
    goal = (4,4)

    maybe_print("\n-- Greedy Best-First (grid) --")
    res = greedy_bfs(start, goal, lambda n: list(grid_neighbors(n, grid)), lambda n: manhattan(n, goal))
    maybe_print("Path:", res['path'])
    maybe_print("Nodes:", res['nodes'], "Time:", round(res['time'],6))

    maybe_print("\n-- A* (grid) --")
    res = a_star(start, goal, lambda n: list(grid_neighbors(n, grid)), lambda n: manhattan(n, goal))
    maybe_print("Path:", res['path'])
    maybe_print("Cost:", res.get('cost'), "Nodes:", res['nodes'], "Time:", round(res['time'],6))

    maybe_print("\n-- RBFS (grid) --")
    res = rbfs(start, goal, lambda n: list(grid_neighbors(n, grid)), lambda n: manhattan(n, goal))
    maybe_print("Path:", res['path'])
    maybe_print("Cost:", res.get('cost'), "Nodes:", res['nodes'], "Time:", round(res['time'],6))

    maybe_print("\n-- A* & RBFS on 8-puzzle (small example) --")
    start8 = (1,2,3,4,5,6,0,7,8)
    ares = a_star(start8, PUZZLE_GOAL, lambda s: list(neighbors_8puzzle(s)), lambda s: manhattan_8p(s))
    rres = rbfs(start8, PUZZLE_GOAL, lambda s: list(neighbors_8puzzle(s)), lambda s: manhattan_8p(s))
    maybe_print("A* path length:", len(ares['path']) if ares['path'] else None, "Nodes:", ares['nodes'])
    maybe_print("RBFS path length:", len(rres['path']) if rres['path'] else None, "Nodes:", rres['nodes'])

    if HAS_NX:
        maybe_print("\n-- networkx A* demo (grid) --")
        nx_path, nx_cost = demo_with_networkx_grid(grid, start, goal)
        maybe_print("networkx A* path:", nx_path, "cost:", nx_cost)


=== Lab 4: library-friendly informed searches ===

-- Greedy Best-First (grid) --

Path:
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]

Nodes: 9 Time: 3.8e-05

-- A* (grid) --

...

-- A* & RBFS on 8-puzzle (small example) --

A* path length: 3 Nodes: 3

RBFS path length: 3 Nodes: 3

-- networkx A* demo (grid) --

networkx A* path:
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
cost: 8

In [5]:
"""
lab5_with_libs.py
Lab 5: IDA*, simplified SMA* — library-first with graceful fallbacks.
Run: python lab5_with_libs.py
Optional: pip install psutil rich tqdm
"""

import time
import heapq
from functools import lru_cache

# optional niceties
try:
    import psutil
    HAS_PSUTIL = True
except Exception:
    HAS_PSUTIL = False

try:
    from rich import print as rprint
    HAS_RICH = True
except Exception:
    HAS_RICH = False

def maybe_print(*args, **kwargs):
    if HAS_RICH:
        rprint(*args, **kwargs)
    else:
        print(*args, **kwargs)

# -----------------------
# Helpers (grid + 8-puzzle)
# -----------------------

def reconstruct(parent, goal):
    path = []
    cur = goal
    while cur is not None:
        path.append(cur)
        cur = parent.get(cur)
    path.reverse()
    return path

def grid_neighbors(pos, grid):
    r,c = pos
    rows, cols = len(grid), len(grid[0])
    for dr,dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        nr, nc = r+dr, c+dc
        if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] == 0:
            yield (nr,nc), 1

def manhattan(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

PUZZLE_GOAL = (1,2,3,4,5,6,7,8,0)
def neighbors_8puzzle(state):
    idx = state.index(0)
    r,c = divmod(idx,3)
    moves=[]
    if r>0: moves.append(-3)
    if r<2: moves.append(3)
    if c>0: moves.append(-1)
    if c<2: moves.append(1)
    for m in moves:
        s=list(state); j=idx+m
        s[idx], s[j] = s[j], s[idx]
        yield tuple(s), 1

def manhattan_8p(state, goal=PUZZLE_GOAL):
    total=0
    for i,v in enumerate(state):
        if v==0: continue
        tr,tc = divmod(i,3)
        gi = goal.index(v)
        gr,gc = divmod(gi,3)
        total += abs(tr-gr)+abs(tc-gc)
    return total

# -----------------------
# IDA* (iterative deepening on f = g + h)
# -----------------------

def ida_star(start, goal, successors_fn, heuristic_fn):
    t0 = time.time()
    nodes_expanded = 0

    def dfs(node, g, threshold, parent, onpath):
        nonlocal nodes_expanded
        f = g + heuristic_fn(node)
        nodes_expanded += 1
        if f > threshold:
            return None, f
        if node == goal:
            return True, g
        min_over = float('inf')
        for nb, cost in successors_fn(node):
            if nb in onpath:
                continue
            parent[nb] = node
            onpath.add(nb)
            found, val = dfs(nb, g + cost, threshold, parent, onpath)
            onpath.remove(nb)
            if found:
                return True, val
            if val < min_over:
                min_over = val
        return None, min_over

    threshold = heuristic_fn(start)
    parent = {start: None}
    while True:
        onpath = set([start])
        found, val = dfs(start, 0, threshold, parent, onpath)
        if found:
            return {'path': reconstruct(parent, goal), 'cost': val, 'nodes': nodes_expanded, 'time': time.time()-t0}
        if val == float('inf'):
            return {'path': None, 'nodes': nodes_expanded, 'time': time.time()-t0}
        threshold = val

# -----------------------
# SMA* simplified
# - keeps nodes dict and min-heap by f
# - when memory limit exceeded, drops worst-leaf and continues
# NOTE: educational simplified version
# -----------------------

def sma_star(start, goal, successors_fn, heuristic_fn, memory_limit=1000):
    t0 = time.time()
    nodes = {}  # node -> {parent,g,f,children,expanded}
    def add_node(n, parent, g):
        nodes[n] = {'parent': parent, 'g': g, 'f': g + heuristic_fn(n), 'children': set(), 'expanded': False}
        if parent in nodes:
            nodes[parent]['children'].add(n)
    add_node(start, None, 0)
    open_heap = [(nodes[start]['f'], start)]
    nodes_expanded = 0

    def drop_worst_leaf():
        leaf=None; maxf=-1
        for n,info in nodes.items():
            if len(info['children'])==0 and n != start:
                if info['f'] > maxf:
                    maxf = info['f']; leaf = n
        if leaf is None: return False
        p = nodes[leaf]['parent']
        if p is not None and leaf in nodes[p]['children']:
            nodes[p]['children'].remove(leaf)
        del nodes[leaf]
        return True

    while open_heap:
        # pop valid entry
        while open_heap and (open_heap[0][1] not in nodes or nodes[open_heap[0][1]]['expanded']):
            heapq.heappop(open_heap)
        if not open_heap:
            break
        f,node = heapq.heappop(open_heap)
        if node not in nodes:
            continue
        nodes[node]['expanded'] = True
        nodes_expanded += 1
        if node == goal:
            parent_map = {n: nodes[n]['parent'] for n in nodes}
            return {'path': reconstruct(parent_map, goal), 'cost': nodes[node]['g'], 'nodes': nodes_expanded, 'time': time.time()-t0}
        for nb, cost in successors_fn(node):
            ng = nodes[node]['g'] + cost
            if nb in nodes:
                if ng < nodes[nb]['g']:
                    nodes[nb]['g'] = ng
                    nodes[nb]['f'] = ng + heuristic_fn(nb)
                    nodes[nb]['parent'] = node
            else:
                add_node(nb, node, ng)
            heapq.heappush(open_heap, (nodes[nb]['f'], nb))
            if len(nodes) > memory_limit:
                dropped = drop_worst_leaf()
                if not dropped:
                    return {'path': None, 'nodes': nodes_expanded, 'time': time.time()-t0, 'note': 'memory too small'}
    return {'path': None, 'nodes': nodes_expanded, 'time': time.time()-t0}

# -----------------------
# Example runs
# -----------------------

if __name__ == "__main__":
    maybe_print("=== Lab 5: IDA* and simplified SMA* ===")
    grid = [
        [0,0,0,1,0],
        [1,1,0,1,0],
        [0,0,0,0,0],
        [0,1,1,1,0],
        [0,0,0,0,0]
    ]
    start = (0,0); goal = (4,4)

    maybe_print("\n-- IDA* (grid) --")
    r = ida_star(start, goal, lambda n: list(grid_neighbors(n, grid)), lambda n: manhattan(n, goal))
    maybe_print("Path:", r['path']); maybe_print("Nodes:", r['nodes'], "Time:", round(r['time'],6))

    maybe_print("\n-- SMA* (grid, mem limit=30) --")
    r = sma_star(start, goal, lambda n: list(grid_neighbors(n, grid)), lambda n: manhattan(n, goal), memory_limit=30)
    maybe_print("Path:", r.get('path')); maybe_print("Nodes:", r['nodes'], "Time:", round(r['time'],6), "Note:", r.get('note'))

    maybe_print("\n-- IDA* (8-puzzle) --")
    start8 = (1,2,3,4,5,6,0,7,8)
    r = ida_star(start8, PUZZLE_GOAL, lambda s: list(neighbors_8puzzle(s)), lambda s: manhattan_8p(s))
    maybe_print("Path length:", len(r['path']) if r['path'] else None, "Nodes:", r['nodes'])

    maybe_print("\n-- SMA* (8-puzzle, mem limit=1000) --")
    r = sma_star(start8, PUZZLE_GOAL, lambda s: list(neighbors_8puzzle(s)), lambda s: manhattan_8p(s), memory_limit=1000)
    maybe_print("Path length:", len(r.get('path')) if r.get('path') else None, "Nodes:", r['nodes'], "Note:", r.get('note'))


=== Lab 5: IDA* and simplified SMA* ===

-- IDA* (grid) --

Path:
[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]

Nodes: 11 Time: 6.5e-05

-- SMA* (grid, mem limit=30) --

...

Nodes: 9 Time: 7.7e-05 Note: None

-- IDA* (8-puzzle) --

Path length: 3 Nodes: 5

-- SMA* (8-puzzle, mem limit=1000) --

Path length: 3 Nodes: 3 Note: None